In [ ]:
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
import arviz as az
from jax import numpy as jnp

import estival.priors as esp
import estival.targets as est

from autumn.core.runs import ManagedRun
from aust_covid.inputs import load_calibration_targets, load_who_data, load_serosurvey_data, get_ifrs
from aust_covid.calibration import get_priors
from emutools.tex import StandardTexDoc
from emutools.inputs import load_param_info
from emutools.calibration import plot_param_progression, view_posterior_comparison, tabulate_param_results
from inputs.constants import PROJECT_PATH, SUPPLEMENT_PATH

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'supplement', "Australia's 2023 Omicron Waves Supplement", 'austcovid')

In [ ]:
case_targets = load_calibration_targets(app_doc)
death_targets = load_who_data(app_doc)
serosurvey_targets = load_serosurvey_data(app_doc)

In [ ]:
param_info = load_param_info(PROJECT_PATH / 'inputs' / 'parameters.yml')
parameters = param_info['value'].to_dict() | get_ifrs(app_doc)

In [ ]:
priors = get_priors()
prior_names = [p.name for p in priors]

In [ ]:
# Set up for calibration or optimisation
def truncation_ceiling(modelled, obs, parameters, time_weights):
    return jnp.where(modelled > obs, -1e11, 0.0)

targets = [
    est.NegativeBinomialTarget('notifications_ma', case_targets, dispersion_param=esp.UniformPrior('notifications_ma_dispersion', (10.0, 140.0))),
    est.NegativeBinomialTarget('deaths_ma', death_targets, dispersion_param=esp.UniformPrior('deaths_ma_dispersion', (60.0, 200.0))),
    est.BinomialTarget('adult_seropos_prop', serosurvey_targets, pd.Series([20] * 4, index=serosurvey_targets.index)),
]
targets.append(est.CustomTarget('seropos_ceiling', pd.Series([0.04], index=[datetime(2021, 12, 1)]), truncation_ceiling, model_key='adult_seropos_prop'))

In [ ]:
run_path = 'projects/aust_covid/base_case_analysis/2023-08-30T1608-try_DEMZ_10k'

In [ ]:
mr = ManagedRun(run_path)
mr.remote.download(mr.remote.list_contents()[-1])
idata = az.from_netcdf(mr.list_local()[-1])

In [ ]:
# Burn
idata = idata.sel(draw=np.s_[2000:])

In [ ]:
app_doc.include_table(tabulate_param_results(idata, priors, param_info), section='Calibration', col_splits=[0.142] * 7, table_width=12.0, longtable=True)

In [ ]:
n_half_priors = round(len(priors) / 2)
plot_param_progression(idata, param_info, app_doc, True, request_vars=prior_names[:n_half_priors], name_ext='_first')

In [ ]:
plot_param_progression(idata, param_info, app_doc, True, request_vars=prior_names[n_half_priors:], name_ext='_last')

In [ ]:
view_posterior_comparison(idata, priors, prior_names[:n_half_priors], param_info['abbreviations'].to_dict(), 0.995, app_doc, name_ext='_first')

In [ ]:
view_posterior_comparison(idata, priors, prior_names[n_half_priors:], param_info['abbreviations'].to_dict(), 0.995, app_doc, name_ext='_last')

In [ ]:
app_doc.write_doc()